In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [8]:
len(pt_features[pt_features['isCase']==False])

10975

In [9]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,non_insomnia_GP_consultations,stroke,CHD_heart_failure_and_peripheral_vascular_disease,hypertension,diabetes,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,antidepressants_pdds:0,antidepressants_pdds:1_10,antidepressants_pdds:11_100,antidepressants_pdds:101_1000,antidepressants_pdds:1001_10000,antidepressants_pdds:above_10000,other_sedatives_pdds:0,other_sedatives_pdds:1_10,other_sedatives_pdds:11_100,other_sedatives_pdds:101_1000,other_sedatives_pdds:1001_10000,other_sedatives_pdds:above_10000,benzo_and_z_drugs_pdds:0,benzo_and_z_drugs_pdds:1_10,benzo_and_z_drugs_pdds:11_100,benzo_and_z_drugs_pdds:101_1000,benzo_and_z_drugs_pdds:1001_10000,benzo_and_z_drugs_pdds:above_10000,mood_stabilisers_pdds:0,mood_stabilisers_pdds:1_10,mood_stabilisers_pdds:11_100,mood_stabilisers_pdds:101_1000,mood_stabilisers_pdds:1001_10000,mood_stabilisers_pdds:above_10000,benzo_and_z_drugs_any,mood_stabilisers_pdds,benzo_and_z_drugs_pdds,other_sedatives_pdds,antidepressants_pdds,insomnia,insomnia_any,insomnia_count:0,insomnia_count:1_10,insomnia_count:above_10,non_insomnia_GP_consultations:0,depot_antipsychotics_pdds,antipsychotics_pdds,antipsychotics_pdds:0,antipsychotics_pdds:1_10,antipsychotics_pdds:11_100,antipsychotics_pdds:101_1000,antipsychotics_pdds:1001_10000,antipsychotics_pdds:above_10000,depot_antipsychotics_pdds:0,depot_antipsychotics_pdds:1_10,depot_antipsychotics_pdds:11_100,depot_antipsychotics_pdds:101_1000,depot_antipsychotics_pdds:1001_10000,depot_antipsychotics_pdds:above_10000
1105,11524120,20,120,1,2005-10-27,True,7323.0,1995-12-12,2005-11-22,16910,85,289,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,24.899708,0.0,58.882554,1,1.0,0.0,1.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5863,4539509,27,509,1,2008-06-11,False,NaN,1999-12-08,2011-06-14,94076,81,68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9090,7298084,18,84,1,2008-10-03,False,NaN,1999-12-10,2011-12-29,21387,90,246,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,19.677601,0.0,234.623291,1,1.0,0.0,1.0,0.0,0.0,0.0,45.701902,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7291,14348463,18,463,1,2007-02-26,True,1350.0,1995-12-13,2007-09-18,69386,89,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6300,8705441,19,441,1,2005-09-29,True,6578.0,1997-08-28,2009-04-03,65105,86,60,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,1.0,0.0,1.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'non_insomnia_GP_consultations', 'stroke',
       'CHD_heart_failure_and_peripheral_vascular_disease', 'hypertension',
       'diabetes', 'mental_illness_non_smi', 'mental_illness_smi',
       'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
       'age_at_index_date:65-69', 'age_at_index_date:70-74',
       'age_at_index_date:75-79', 'age_at_index_date:80-84',
       'age_at_index_date:85-89', 'age_at_index_date:90-99',
       'age_at_index_date:above_99', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'antidepressants_pdds:0',
       'antidepressants_pdds:1_10', 'antidepressants_pdds:11_100',
       'antidepressants_pdds:101_1000', 'antidepressants_pdds:1001_10000',
       'anti

In [16]:
characteristics = [column for column in pt_features.columns if column not in [
    'patid',  'yob', 'pracid', 'index_date', 'isCase',
    'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [17]:
baseline_characteristics = pd.DataFrame(index=characteristics,columns=['Cases','Controls','P value'])

In [18]:
baseline_dichot,baseline_contin = add_baseline_characteristics(baseline_characteristics,characteristics,pt_features)

In [19]:
# baseline_contin

In [20]:
baseline_dichot

,Cases,Controls,P value
CHD_heart_failure_and_peripheral_vascular_disease,2755 (25.1%),2500 (22.8%),0.000
age_at_index_date:65-69,440 (4.0%),440 (4.0%),1.000
age_at_index_date:70-74,1038 (9.5%),1038 (9.5%),1.000
age_at_index_date:75-79,2140 (19.5%),2140 (19.5%),1.000
age_at_index_date:80-84,3055 (27.8%),3055 (27.8%),1.000
age_at_index_date:85-89,2832 (25.8%),2832 (25.8%),1.000
age_at_index_date:90-99,1187 (10.8%),1187 (10.8%),1.000
age_at_index_date:above_99,19 (0.2%),19 (0.2%),1.000
antidepressants_pdds:0,9298 (84.7%),9801 (89.3%),0.000
antidepressants_pdds:1001_10000,366 (3.3%),222 (2.0%),0.000
